In [ ]:
# !pip install torch==2.5.0
# !pip install numpy==1.26.4
# !pip install openai==1.79.0
# !pip install tenacity==9.1.2
# !pip install tiktoken==0.9.0
# !pip install transformers==4.51.3
# !pip install pandas==2.2.3
# !pip install datasets==3.6.0
# !pip install sentencepiece==0.2.0
# !pip install peft==0.15.2
# !pip install evaluate==0.4.3
# !pip install trl==0.11.4
# !pip install protobuf==6.31.0
# !pip install python-dotenv==1.1.0
# !pip install accelerate==1.7.0
# !pip install ipywidgets
# !pip install pynvml==8.1.7
# !pip uninstall torch torchvision torchaudio -y
# !pip install torch==2.5.0 torchvision==0.20.0 torchaudio==2.5.0 --index-url https://download.pytorch.org/whl/cu124
# !pip install --upgrade setuptools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.4 MB/s eta 0:00:00ta 0:00:01


In [4]:
import pkg_resources

libs = [
    "numpy", "openai", "tenacity", "tiktoken", "transformers", "pandas",
    "scikit-learn", "torch", "bitsandbytes", "datasets", "sentencepiece",
    "peft", "evaluate", "trl", "protobuf", "python-dotenv", "pandas_ta",
    "ollama", "accelerate", "ipywidgets"
]

for lib in libs:
    try:
        version = pkg_resources.get_distribution(lib).version
        print(f"{lib}=={version}")
    except pkg_resources.DistributionNotFound:
        print(f"{lib} not installed")

numpy==1.26.4
openai==1.79.0
tenacity==9.1.2
tiktoken==0.9.0
transformers==4.51.3
pandas==2.2.3
scikit-learn not installed
torch==2.5.0+cu124
bitsandbytes not installed
datasets==3.6.0
sentencepiece==0.2.0
peft==0.15.2
evaluate==0.4.3
trl==0.11.4
protobuf==6.31.0
python-dotenv==1.1.0
pandas_ta not installed
ollama not installed
accelerate==1.7.0
ipywidgets==8.1.7


/tmp/ipykernel_3885/316775683.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [5]:
import torch # type: ignore
print(torch.cuda.is_available())  # Nếu trả về False, CUDA chưa hoạt động
print(torch.cuda.device_count())  # Kiểm tra số lượng GPU
print(torch.cuda.get_device_name(0))  # Hiển thị tên GPU
# print(torch.set_default_device())

True
1
GRID P40-24Q


In [7]:
from datasets import load_dataset, Dataset
import re


data_path = "./datasets/"

ds_dict = load_dataset(data_path)

ds_dict

# Lấy tập train từ DatasetDict
ds = ds_dict["train"]


# Tách completion_a và completion_b thành từng dòng mới, rồi tách "Explanation:"
def split_completion(completion_text):
    if "Explanation:" in completion_text:
        parts = completion_text.split("Explanation:", 1)
        target = parts[0].strip()
        explain = "Explanation: " + parts[1].strip()

        explain = explain.replace("\n", "")

    else:
        target = completion_text.strip()
        explain = ""
    return target, explain


# Bước 2: Chuyển đổi từng dòng thành 2 dòng mới
new_data = []
for example in ds:
    prediction_a, explain_a = split_completion(example["completion_a"])
    new_data.append({"user_input": example["user_input"], "completion": example["completion_a"], "prediction": prediction_a, "explain": explain_a})

    prediction_b, explain_b = split_completion(example["completion_b"])
    new_data.append({"user_input": example["user_input"], "completion": example["completion_b"], "prediction": prediction_b, "explain": explain_b})

# Bước 3: Tạo dataset mới từ danh sách đã chuyển đổi
new_ds = Dataset.from_list(new_data)

# Bước 4: In thử dữ liệu
print(new_ds[1]['explain'])
new_ds[1]['explain']
new_ds

Explanation: Despite negative news such as the ban on N95 and surgical masks for the general public and criticisms of being a sweatshop, Amazon's fulfillment of 80,000 out of 100,000 jobs to meet increased demand during the pandemic and Jeff Bezos' donation to Feeding America showcase their commitment to community support. Partnering with SXSW for an online festival and planning to provide masks and temperature checks for staff demonstrate adaptability. With news of Canada's agreement with Amazon Canada and BMO Capital Markets' "Outperform" rating, there are positive signals. Speculations about Amazon's Prime Day event postponement may have led to short-term stock fluctuations, but the overall sentiment remains optimistic.From a technical perspective, despite the daily stock price range and fluctuation, key indicators such as the ADX and RSI suggest a strong bullish momentum. The contact with coronavirus test makers and anticipation for advanced technological solutions like driverless 

Dataset({
    features: ['user_input', 'completion', 'prediction', 'explain'],
    num_rows: 6
})

In [ ]:
LLM_GUIDANCE_MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

In [2]:
# !pip install torchdiffeq

# Guidance and FlowModel

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from transformers import AutoModelForCausalLM, AutoTokenizer
from torchdiffeq import odeint  # Cần cài đặt: pip install torchdiffeq
import numpy as np
import random

# Thiết lập seed
fix_seed = 100
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LLM_GUIDANCE_MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

MAX_SEQ_LENGTH_GUIDANCE = 1024
LEARNING_RATE_FLOW_MODEL = 2e-4

# Lớp Guidance LLM hướng đối tượng
class GuidanceLLM:
    def __init__(self, model_name=LLM_GUIDANCE_MODEL_NAME):
        self.model_name = model_name
        self.tokenizer = None
        self.model = None
        self.hidden_dim = None
        self.load_model()

    def load_model(self):
        """Tải mô hình và tokenizer"""
        print(f"Loading Guidance LLM: {self.model_name}...")
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, trust_remote_code=True)
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.bfloat16,
                trust_remote_code=True
            ).to(DEVICE)
            
            for param in self.model.parameters():
                param.requires_grad = False
            self.model.eval()

            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
                self.model.config.pad_token_id = self.model.config.eos_token_id

            self.hidden_dim = self.model.config.hidden_size
            print(f"Guidance LLM loaded on {DEVICE}. Hidden dim: {self.hidden_dim}")
        except Exception as e:
            print(f"Error loading model: {e}")
            raise

    def get_decision_distribution(self, user_input_text, explanation_sentences_so_far, all_possible_actions):
        """Tính phân phối xác suất cho các hành động"""
        current_explanation = " ".join(explanation_sentences_so_far)
        action_logits_list = []

        with torch.no_grad():
            for action_text in all_possible_actions:
                prompt = (
                    f"Given the following information about a company: '{user_input_text}'.\n"
                    f"Based on the reasoning: '{current_explanation}'.\n"
                    f"Predict the price impact: {action_text}."
                )
                inputs = self.tokenizer(
                    prompt,
                    return_tensors="pt",
                    padding=True,
                    truncation=True,
                    max_length=MAX_SEQ_LENGTH_GUIDANCE
                ).to(DEVICE)

                try:
                    outputs = self.model(**inputs)
                    logits = outputs.logits[:, -1, :]  # Logits token cuối
                    action_tokens = self.tokenizer(action_text, add_special_tokens=False).input_ids
                    if not action_tokens:
                        action_logits_list.append(torch.tensor(-10.0).to(DEVICE))
                        continue

                    # Giả lập: Trung bình logits (cần fine-tune thực tế)
                    logit_score = logits.mean()
                    action_logits_list.append(logit_score)
                except Exception as e:
                    print(f"Error processing action '{action_text}': {e}")
                    action_logits_list.append(torch.tensor(-10.0).to(DEVICE))

        if not action_logits_list or all(l.item() == -10.0 for l in action_logits_list):
            return torch.ones(len(all_possible_actions), device=DEVICE) / len(all_possible_actions)

        logits_tensor = torch.stack(action_logits_list)
        probability_distribution = torch.softmax(logits_tensor, dim=0)
        return probability_distribution

    def get_last_layer_hidden_states(self, user_input_text, explanation_sentences_so_far, all_possible_actions_text_for_prompt):
        """Lấy hidden states từ tầng cuối"""
        current_explanation = " ".join(explanation_sentences_so_far)
        prompt = (
            f"Given the following information about a company: '{user_input_text}'.\n"
            f"Based on the reasoning: '{current_explanation}'.\n"
            f"Consider the possible outcomes: {all_possible_actions_text_for_prompt}."
        )

        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=MAX_SEQ_LENGTH_GUIDANCE
        ).to(DEVICE)

        with torch.no_grad():
            outputs = self.model(**inputs, output_hidden_states=True)
        return outputs.hidden_states[-1]  # [batch_size, seq_len, hidden_dim]

# Mô hình Rectified Flow
class RectifiedFlowModel(nn.Module):
    def __init__(
        self,
        guidance_llm_instance,  # Instance của GuidanceLLM
        num_actions,
        flow_embed_dim=256,
        projector_hidden_dim=256,
        projector_layers=4,
        num_attention_heads=4,
        dropout_rate=0.1
    ):
        super().__init__()
        self.guidance_llm_instance = guidance_llm_instance
        self.num_actions = num_actions
        self.flow_embed_dim = flow_embed_dim
        self.projector_hidden_dim = projector_hidden_dim
        self.projector_layers = projector_layers
        self.num_attention_heads = num_attention_heads

        # Kích thước hidden state từ Guidance LLM
        self.hidden_dim_guidance_llm = self.guidance_llm_instance.hidden_dim

        # Embedding cho z_t và t
        self.zt_embed_mlp = nn.Sequential(
            nn.Linear(num_actions, flow_embed_dim),
            nn.ReLU(),
            nn.LayerNorm(flow_embed_dim)
        )
        self.time_embed_mlp = nn.Sequential(
            nn.Linear(1, flow_embed_dim),
            nn.ReLU(),
            nn.LayerNorm(flow_embed_dim)
        )

        # Truy cập W_Q, W_K, W_V từ Guidance LLM
        try:
            # Qwen-1.5B: transformer.h[-1].attn.q_proj/k_proj/v_proj
            last_guidance_layer_attn = self.guidance_llm_instance.model.transformer.h[-1].attn
            self.guidance_wq = last_guidance_layer_attn.q_proj
            self.guidance_wk = last_guidance_layer_attn.k_proj
            self.guidance_wv = last_guidance_layer_attn.v_proj
            print("Successfully accessed Q, K, V projection weights from Guidance LLM.")

            # Đóng băng trọng số
            for param in [self.guidance_wq.parameters(), self.guidance_wk.parameters(), self.guidance_wv.parameters()]:
                for p in param:
                    p.requires_grad = False
            self.using_borrowed_weights = True
        except AttributeError as e:
            print(f"Warning: Could not access Q, K, V weights ({e}). Using new Linear layers.")
            self.flow_qkv_proj = nn.Linear(flow_embed_dim, self.hidden_dim_guidance_llm)
            self.guidance_kv_proj = nn.Linear(self.hidden_dim_guidance_llm, self.hidden_dim_guidance_llm)
            self.using_borrowed_weights = False

        # Cross-attention
        self.attention_layer = nn.MultiheadAttention(
            embed_dim=self.hidden_dim_guidance_llm,
            num_heads=num_attention_heads,
            dropout=dropout_rate,
            batch_first=True
        )
        self.attn_output_norm = nn.LayerNorm(self.hidden_dim_guidance_llm)
        self.attn_output_projection = nn.Linear(self.hidden_dim_guidance_llm, flow_embed_dim)

        # Projector MLP
        projector_modules = []
        current_dim = flow_embed_dim
        for i in range(projector_layers):
            output_dim = projector_hidden_dim if i < projector_layers - 1 else num_actions
            projector_modules.append(nn.Linear(current_dim, output_dim))
            if i < projector_layers - 1:
                projector_modules.extend([
                    nn.ReLU(),
                    nn.LayerNorm(output_dim),
                    nn.Dropout(dropout_rate)
                ])
                current_dim = output_dim
        self.projector_mlp = nn.Sequential(*projector_modules)

        self.optimizer = optim.AdamW(self.parameters(), lr=LEARNING_RATE_FLOW_MODEL)

    def _project_for_attention(self, tensor, projection_type):
        """Áp dụng phép chiếu Q, K, V"""
        if self.using_borrowed_weights:
            if projection_type == 'q': return self.guidance_wq(tensor)
            elif projection_type == 'k': return self.guidance_wk(tensor)
            elif projection_type == 'v': return self.guidance_wv(tensor)
        else:
            if projection_type == 'q': return self.flow_qkv_proj(tensor)
            elif projection_type in ['k', 'v']: return self.guidance_kv_proj(tensor)
        raise ValueError(f"Unknown projection_type: {projection_type}")

    def forward(self, z_t, time_t, guidance_llm_last_hidden_states):
        batch_size = z_t.shape[0]

        # Embedding z_t và t
        h_emb_zt = self.zt_embed_mlp(z_t)  # [batch_size, flow_embed_dim]
        h_emb_t = self.time_embed_mlp(time_t)  # [batch_size, flow_embed_dim]
        flow_tokens_emb = (h_emb_zt + h_emb_t).unsqueeze(1)  # [batch_size, 1, flow_embed_dim]

        # Cross-attention
        query = self._project_for_attention(flow_tokens_emb, 'q')  # [batch_size, 1, hidden_dim_guidance_llm]
        key = self._project_for_attention(guidance_llm_last_hidden_states, 'k')  # [batch_size, seq_len, hidden_dim_guidance_llm]
        value = self._project_for_attention(guidance_llm_last_hidden_states, 'v')  # [batch_size, seq_len, hidden_dim_guidance_llm]

        attn_output, _ = self.attention_layer(query, key, value, need_weights=False)
        attn_output_normalized = self.attn_output_norm(attn_output)
        h_attn_zt = self.attn_output_projection(attn_output_normalized.squeeze(1))  # [batch_size, flow_embed_dim]

        # Projector
        vector_field = self.projector_mlp(h_attn_zt)  # [batch_size, num_actions]
        return vector_field

    def train_step(self, z0_batch, z1_batch, time_batch, guidance_llm_hidden_states_batch):
        self.optimizer.zero_grad()
        z_t_batch = time_batch * z1_batch + (1.0 - time_batch) * z0_batch
        target_vector_field = z1_batch - z0_batch
        predicted_vector_field = self.forward(z_t_batch, time_batch, guidance_llm_hidden_states_batch)
        loss = nn.MSELoss()(predicted_vector_field, target_vector_field)
        loss.backward()
        self.optimizer.step()
        return loss.item()

    @torch.no_grad()
    def solve_ode_generate_distribution(self, z0, guidance_llm_last_hidden_states, num_steps=10, return_logits=False):
        self.eval()
        if z0.ndim == 1: z0 = z0.unsqueeze(0)
        if guidance_llm_last_hidden_states.ndim == 2: guidance_llm_last_hidden_states = guidance_llm_last_hidden_states.unsqueeze(0)
        batch_size = z0.shape[0]

        def ode_func(t, z):
            time_t = torch.full((batch_size, 1), t, device=DEVICE)
            return self.forward(z, time_t, guidance_llm_last_hidden_states)

        t = torch.linspace(0, 1, num_steps, device=DEVICE)
        z_t = odeint(ode_func, z0, t, method='rk4')  # Runge-Kutta
        z1_hat = z_t[-1]
        output = z1_hat.squeeze(0) if batch_size == 1 else z1_hat
        self.train()
        return output if return_logits else torch.softmax(output, dim=-1)

# --- Test tích hợp ---
if __name__ == '__main__':
    # Khởi tạo Guidance LLM
    guidance_llm = GuidanceLLM()

    # Dữ liệu mẫu
    sample_data = {
        'user_input': "Công ty Alpha vừa ra mắt sản phẩm mới X được thị trường đón nhận nồng nhiệt. Doanh số dự kiến tăng 30% trong quý tới. Tuy nhiên, chi phí R&D cho sản phẩm Y đang tăng cao.",
        'prediction': "positive",
        'explain': "Sản phẩm X thành công rực rỡ là yếu tố chính. Nó bù đắp được lo ngại về chi phí R&D cho Y. Triển vọng tăng trưởng doanh số rất khả quan."
    }
    POSSIBLE_ACTIONS = ["negative", "positive"]
    correct_action = sample_data['prediction']
    correct_action_idx = POSSIBLE_ACTIONS.index(correct_action)

    # Tách giải thích
    explanation_sentences = [s.strip() for s in sample_data['explain'].split('.') if s.strip()]
    print("Explanation Sentences:", explanation_sentences)

    # Khởi tạo Rectified Flow
    flow_model = RectifiedFlowModel(
        guidance_llm_instance=guidance_llm,
        num_actions=len(POSSIBLE_ACTIONS),
        flow_embed_dim=128,
        projector_hidden_dim=128,
        num_attention_heads=2
    ).to(DEVICE)

    # Test Guidance LLM
    prob_distributions = []
    hidden_states_list = []
    for k in range(len(explanation_sentences) + 1):
        current_explanation = explanation_sentences[:k]
        prob_dist = guidance_llm.get_decision_distribution(
            sample_data['user_input'], current_explanation, POSSIBLE_ACTIONS
        )
        hidden_states = guidance_llm.get_last_layer_hidden_states(
            sample_data['user_input'], current_explanation, ", ".join(POSSIBLE_ACTIONS)
        )
        prob_distributions.append(prob_dist)
        hidden_states_list.append(hidden_states)
        print(f"\nProbability Distribution after {k} sentences: {prob_dist.tolist()}")
        print(f"  Negative: {prob_dist[0]:.4f}, Positive: {prob_dist[1]:.4f}")
        print(f"  Hidden States Shape: {hidden_states.shape}")

    # Test Rectified Flow
    batch_size = 1
    z0_test = torch.randn(batch_size, len(POSSIBLE_ACTIONS)).to(DEVICE)
    time_test = torch.rand(batch_size, 1).to(DEVICE)
    guidance_hs_test = hidden_states_list[-1]  # Hidden states từ câu cuối
    vector_field = flow_model(z0_test, time_test, guidance_hs_test)
    print(f"\nVector Field Shape: {vector_field.shape}")
    assert vector_field.shape == (batch_size, len(POSSIBLE_ACTIONS))

    # Test ODE solver
    p_hat_dist = flow_model.solve_ode_generate_distribution(z0_test, guidance_hs_test, num_steps=5)
    print(f"p_hat Distribution: {p_hat_dist.shape}, Sum: {p_hat_dist.sum()}")
    assert p_hat_dist.shape == (batch_size, len(POSSIBLE_ACTIONS))
    assert torch.isclose(p_hat_dist.sum(), torch.tensor(1.0, device=DEVICE), atol=1e-5)

    # Tính phần thưởng
    flow_distributions = []
    for k in range(len(explanation_sentences) + 1):
        flow_dist = flow_model.solve_ode_generate_distribution(z0_test, hidden_states_list[k], num_steps=5)
        flow_distributions.append(flow_dist)
    rewards = [flow_distributions[k][correct_action_idx] - flow_distributions[k-1][correct_action_idx] 
               for k in range(1, len(flow_distributions))]
    print(f"\nRewards: {rewards}")

    print("Integration tests passed.")

# demo

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from torchdiffeq import odeint
from trl import PPOTrainer, PPOConfig
from peft import LoraConfig
from datasets import load_dataset, concatenate_datasets
from tqdm import tqdm
import numpy as np
import random
import os

# Thiết lập seed
fix_seed = 100
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LLM_MODEL_NAME = "Qwen/Qwen1.5-1.8B"  # Thay DeepSeek-R-1.5B
MAX_SEQ_LENGTH = 1024
LEARNING_RATE_FLOW_MODEL = 2e-4

# Lớp Explanation LLM
class ExplanationLLM:
    def __init__(self, model_name=LLM_MODEL_NAME):
        self.model_name = model_name
        self.tokenizer = None
        self.model = None
        self.load_model()

    def load_model(self):
        print(f"Loading Explanation LLM: {self.model_name}...")
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, trust_remote_code=True)
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.bfloat16,
                trust_remote_code=True
            ).to(DEVICE)
            self.model.eval()
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
                self.model.config.pad_token_id = self.model.config.eos_token_id
            print(f"Explanation LLM loaded on {DEVICE}.")
        except Exception as e:
            print(f"Error loading model: {e}")
            raise

    def generate_explanation(self, context):
        """Tạo giải thích cho ngữ cảnh"""
        prompt = f"Analyze: {context}. Provide reasoning without predicting outcome."
        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=MAX_SEQ_LENGTH
        ).to(DEVICE)
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_length=100,
                pad_token_id=self.tokenizer.pad_token_id,
                eos_token_id=self.tokenizer.eos_token_id
            )
        explanation = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return [s.strip() for s in explanation.split(". ") if s.strip()]

# Lớp Guidance LLM
class GuidanceLLM:
    def __init__(self, model_name=LLM_MODEL_NAME):
        self.model_name = model_name
        self.tokenizer = None
        self.model = None
        self.hidden_dim = None
        self.load_model()

    def load_model(self):
        print(f"Loading Guidance LLM: {self.model_name}...")
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, trust_remote_code=True)
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.bfloat16,
                trust_remote_code=True
            ).to(DEVICE)
            for param in self.model.parameters():
                param.requires_grad = False
            self.model.eval()
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
                self.model.config.pad_token_id = self.model.config.eos_token_id
            self.hidden_dim = self.model.config.hidden_size
            print(f"Guidance LLM loaded on {DEVICE}. Hidden dim: {self.hidden_dim}")
        except Exception as e:
            print(f"Error loading model: {e}")
            raise

    def get_decision_distribution(self, user_input_text, explanation_sentences_so_far, all_possible_actions):
        current_explanation = " ".join(explanation_sentences_so_far)
        action_logits_list = []
        with torch.no_grad():
            for action_text in all_possible_actions:
                prompt = (
                    f"Given the following information about a company: '{user_input_text}'.\n"
                    f"Based on the reasoning: '{current_explanation}'.\n"
                    f"Predict the price impact: {action_text}."
                )
                inputs = self.tokenizer(
                    prompt,
                    return_tensors="pt",
                    padding=True,
                    truncation=True,
                    max_length=MAX_SEQ_LENGTH
                ).to(DEVICE)
                try:
                    outputs = self.model(**inputs)
                    logits = outputs.logits[:, -1, :]
                    action_tokens = self.tokenizer.action_text, return_tensors="pt", add_special_tokens=False).input_ids
                    if not action_tokens:
                        action_logits_list.append(torch.tensor(-10.0).to(DEVICE))
                        continue
                    logit_score = logits.mean()  # Giả lập, cần fine-tune
                    action_logits_list.append(logit_score)
                except Exception as e:
                    print(f"Error processing action '{action_text}': {e}")
                    action_logits_list.append(torch.tensor(-10.0).to(device))
        if not action_logits_list or all(l.item() == -10.0 for l in action_logits_list):
            return torch.ones(len(all_possible_actions), device=DEVICE) / len(all_possible_actions))
        logits_tensor = torch.stack(action_logits_list)
        probability_distribution = torch.softmax(logits_tensor, dim=0)
        return probability_distribution

    def get_last_layer_hidden_states(self, user_input_text, explanation_sentences_so_far, all_possible_actions_text_for_prompt):
        current_explanation = " ".join(explanation_sentences_so_far)
        prompt = (
            f"Given the following information about a company: '{user_input_text}'.\n"
            f"Based on the reasoning: '{current_explanation}'.\n"
            f"Consider the possible outcomes: {all_possible_actions_text_for_prompt}."
        )
        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=MAX_MAC_SEQ_LENGTH
        ).to(device)
        with torch.no_grad():
            outputs = self.model(**inputs, output_hidden_states=True)
        return outputs.hidden_states[-1]  # [batch_size, seq_len, hidden_dim]

# Mô hình Rectified Flow
class RectifiedFlowModel(nn.Module):
    def __init__(
        self,
        guidance_llm_instance,
        num_actions,
        flow_embed_dim=256,
        projector_hidden_dim=256,
        num_attention_heads=4,
        dropout_rate=0.1
    ):
        super().__init__()
        self.guidance_llm_instance = guidance_llm_instance
        self.num_actions = num_actions
        self.flow_embed_dim = flow_embed_dim
        self.projector_hidden_dim = projector_hidden_dim
        self.num_attention_heads = num_attention_heads

        # Kích thước từ Guidance LLM
        self.hidden_dim_guidance_llm = self.guidance_llm_instance.hidden_dim

        # Embedding
        self.zt_embed_mlp = nn.Sequential(
            nn.Linear(num_actions, flow_embed_dim),
            nn.ReLU(),
            nn.LayerNorm(flow_embed_dim)
        )
        self.time_embed_mlp = nn.Sequential(
            nn.Linear(1, flow_embed_dim),
            nn.ReLU(),
            nn.LayerNorm(flow_embed_dim)
        )

        # Truy cập W_Q, W_K, W_V
        try:
            last_guidance_layer_attn = self.guidance_guidance_instance.model.transformer.h[-1].attn
            self.guidance_wq = last_guidance_layer_attn.q_wq
            self.guidance_wk = last_guidance_layer_attn.k_wk
            self.guidance_wv = last_guidance_layer_attn.v_wv
            print("Successfully accessed Q, K, V projection weights from Guidance LLM.")
            for param in [self.guidance_wq.parameters(), self_guidance_wk.parameters(), self_guidance_wv.parameters()]:
                for p in param:
                    p.requires_grad = False
            self.using_borrowed_weights = True
        except AttributeError as e:
            print(f"Warning: Could not access Q, K, V weights ({e}). Using new Linear layers.")
            self.flow_qkv_proj = nn.Linear(self.flow_embed_dim, self.hidden_dim_guidance_llm)
            self.guidance_kv_proj = nn.Linear(self.hidden_dim_guidance_llm, self.hidden_dim_guidance_llm)
            self.using_borrowed_weights = False

        # Cross-attention
        self.attention_layer = nn.MultiheadAttention(
            embed_dim=self.hidden_dim_guidance_guidance_llm,
            num_heads=num_attention_heads,
            dropout=dropout_rate,
            batch_first=True
        )
        self.attn_output_norm = nn.LayerNorm(self.hidden_dim_guidance_llm)
        self.attn_output_projection = nn.Linear(self.hidden_dim_guidance_llm, self.flow_embed_dim))

        # Projector MLP
        projector_modules = []
        current_dim = flow_embed_dim
        for i in range(projector_layers):
            output_dim = projector_hidden_dim if i < projector_layers - 1 else num_actions
            projector_modules.append(nn.Linear(current_dim, output_dim))
            if i < projector_layers - 1:
                projector_modules.extend([
                    nn.ReLU(),
                    nn.LayerNorm(output_dim),
                    nn.Dropout(dropout_rate)
                ])
                current_dim = output_dim
        self.projector_mlp = nn.Sequential(*projector_modules)

        self.optimizer = optim.AdamW(self.parameters(), lr=LEARNING_RATE_FLOW_MODEL)

    def _project_for_attention(self, tensor, projection_type):
        if self.using_borrowed_weights:
            if projection_type == 'q': return self.guidance_wq(tensor)
            elif projection_type == 'k': return self.guidance_wk(tensor)
            elif projection_type == 'v': return self_guidance_wv(tensor)
        else:
            if projection_type == 'q': return self.flow_qkv_qkv(tensor)
            elif projection_type in ['k', 'v']: return self_guidance_kv_proj(tensor)
        raise ValueError(f"Unknown projection_type: {projection_type}")

    def forward(self, z_t, time_t, guidance_llm_last_hidden_states):
        batch_size = z_t.shape[0]
        # Embedding
        h_emb_zt = self.zt_embed_mlp(z_t)
        h_emb_t = self.time_embed_mlp(time_t)
        flow_tokens_emb = (h_emb_zt + h_emb_t).unsqueeze(1)

        # Cross-attention
        query = self._project_for_attention(flow_tokens_emb, 'q')
        key = self._project_for_attention(guidance_llm_last_hidden_states, 'k')
        value = self._project_for_attention(guidance_lm_last_hidden_states, 'v')
        attn_output, _ = self.attention_layer(query, key, value, need_weights=False)
        attn_output_normalized = self.attn_output_norm(attn_output)
        h_attn_zt = self.attn_output_projection(attn_output_normalized.squeeze(1)))

        # Projector
        vector_field = self.projector_mlp(h_attn_zt)
        return vector_field

    def train_step(self, z0_batch, z1_batch, time_batch, guidance_llm_hidden_states_batch):
        self.optimizer.zero_grad()
        z_t_batch = time_batch * z1_batch + (1.0 - time_batch) * z0_batch
        target_vector_field = z1_batch - z0_batch
        predicted_vector_field = self.forward(z_t_batch, time_batch, guidance_llm_hidden_states_batch)
        loss = nn.MSELoss()(predicted_vector_field, target_vector_field)
        loss.backward()
        self.optimizer.step()
        return loss.item()

    @torch.no_grad()
    def solve_ode_generate_distribution(self, z0, guidance_llm_last_hidden_states, num_steps=10, return_logits=False):
        self.eval()
        if z0.ndim == 1: z0 = z0.unsqueeze(0)
        if guidance_llm_last_hidden_states.ndim == 2: guidance_llm_last_hidden_states = guidance_llm_last_hidden_states.unsqueeze(0)
        batch_size = z0.shape[0]
        def ode_func(t, z):
            time_t = torch.full((batch_size, 1), t, device=DEVICE)
            return self.forward(z, time_t, guidance_llm_last_hidden_states)
        t = torch.linspace(0, 1, num_steps, device=DEVICE)
        z_t = odeint(ode_func, z0, t, method='rk4')
        z1_hat = z_t[-1]
        output = z1_hat.squeeze(0) if batch_size == 1 else z1_hat
        self.train()
        return output if return_logits else torch.softmax(output, dim=-1)

# PPO với LoRA
def tuning_lm_with_rl(args):
    # Cấu hình PPO
    config = PPOConfig(
        learning_rate=args.rl_learning_rate,
        batch_size=4,
        mini_batch_size=2,
        gradient_accumulation_steps=1,
        ppo_epochs=args.ppo_epochs,
        seed=args.seed,
    )

    # Tải dataset
    dataset_name = args.datasets_dir
    train_dataset = load_dataset(dataset_name, split="train")
    print(f"Dataset size: {len(train_dataset)}")

    # Tải tokenizer
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    def build_dataset(tokenizer, dataset_name):
        """Tạo dataset cho PPO"""
        ds = load_dataset(dataset_name, split="train")
        repeat_factor = 64 // len(ds) + 1
        ds = concatenate_datasets([ds] * repeat_factor).select(range(64))
        
        def preprocess_function(examples):
            new_examples = {
                "query": [],
                "input_ids": [],
                "context": [],
                "label": []
            }
            for context, label in zip(examples["context"], examples["label"]):
                query = f"Analyze: {context}. Provide reasoning without predicting outcome."
                tokenized_query = tokenizer(
                    query,
                    truncation=True,
                    max_length=512,
                    return_tensors="pt"
                )
                new_examples["query"].append(query)
                new_examples["input_ids"].append(tokenized_query["input_ids"].squeeze(0))
                new_examples["context"].append(context)
                new_examples["label"].append(label)
            return new_examples

        ds = ds.map(preprocess_function, batched=True, remove_columns=ds.column_names)
        ds.set_format(type="torch")
        return ds

    dataset = build_dataset(tokenizer, dataset_name)
    print(f"Dataset created with {len(dataset)} samples")

    def collator(data):
        return dict((key, [d[key] for d in data]) for key in data[0])

    # Cấu hình LoRA
    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
    )

    # Tải mô hình với LoRA
    from transformers import AutoModelForCausalLMWithValueHead
    model = AutoModelForCausalLMWithValueHead.from_pretrained(
        args.rl_base_model,
        torch_dtype=torch.bfloat16,
        peft_config=lora_config,
        trust_remote_code=True
    )
    model.base_model_prefix = "model"
    model.generation_config = GenerationConfig.from_pretrained(args.rl_base_model, trust_remote_code=True)
    print(f"Finetune model: {args.rl_base_model}")

    # Khởi tạo PPOTrainer
    ppo_trainer = PPOTrainer(
        config=config,
        model=model,
        tokenizer=tokenizer,
        dataset=dataset,
        data_collator=collator
    )

    # Khởi tạo ExplanationLLM, GuidanceLLM, RectifiedFlowModel
    explanation_llm = ExplanationLLM()
    guidance_llm = GuidanceLLM()
    flow_model = RectifiedFlowModel(
        guidance_llm_instance=guidance_llm,
        num_actions=2,  # negative, positive
        flow_embed_dim=128,
        projector_hidden_dim=128,
        num_attention_heads=2
    ).to(DEVICE)

    # Cấu hình sinh văn bản
    generation_kwargs = {
        "top_k": 0.0,
        "top_p": 1.0,
        "do_sample": True,
        "pad_token_id": tokenizer.pad_token_id,
        "eos_token_id": tokenizer.eos_token_id,
        "max_length": args.output_max_length
    }

    # Vòng lặp huấn luyện PPO
    for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
        query_tensors = batch["input_ids"]
        contexts = batch["context"]
        labels = batch["label"]

        # Sinh phản hồi từ mô hình PPO (ExplanationLLM với LoRA)
        response_tensors = ppo_trainer.generate(
            query_tensors,
            return_prompt=False,
            **generation_kwargs
        )
        responses = tokenizer.batch_decode(response_tensors, skip_special_tokens=True)
        batch["response"] = [r.split(". ") for r in responses if r.strip()]

        # Tính phần thưởng từ RectifiedFlowModel
        rewards = []
        for context, response, label in zip(contexts, batch["response"], labels):
            prob_distributions = []
            hidden_states_list = []
            z0 = torch.randn(1, 2).to(DEVICE)  # 2 actions
            for k in range(len(response) + 1):
                current_explanation = response[:k]
                hidden_states = guidance_llm.get_last_layer_hidden_states(
                    context, current_explanation, "negative, positive"
                )
                flow_dist = flow_model.solve_ode_generate_distribution(
                    z0, hidden_states, num_steps=5
                )
                prob_distributions.append(flow_dist)
                hidden_states_list.append(hidden_states)
            correct_idx = 1 if label == "positive" else 0
            reward = sum(
                prob_distributions[k][correct_idx] - prob_distributions[k-1][correct_idx]
                for k in range(1, len(prob_distributions))
            )
            rewards.append(reward)

        # Bước PPO
        stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
        ppo_trainer.log_stats(stats, batch, rewards)

        # Lưu checkpoint
        if args.save_freq and epoch and epoch % args.save_freq == 0:
            save_dir = os.path.join(args.output_dir, f"step_{epoch}")
            ppo_trainer.save_pretrained(save_dir)
            print(f"Saved checkpoint at: {save_dir}")

    # Lưu checkpoint cuối
    final_save_dir = os.path.join(args.output_dir, "step_final")
    ppo_trainer.save_pretrained(final_save_dir)
    print(f"Final checkpoint saved at: {final_save_dir}")

# --- Test tích hợp ---
if __name__ == '__main__':
    from dataclasses import dataclass

    @dataclass
    class Args:
        rl_learning_rate = 2e-5
        ppo_epochs = 10
        seed = 100
        datasets_dir = "path/to/your/dataset"  # Thay bằng dataset thực tế
        tokenizer_name = LLM_MODEL_NAME
        rl_base_model = LLM_MODEL_NAME
        output_max_length = 100
        save_freq = 5
        output_dir = "./ppo_checkpoints"

    args = Args()

    # Giả lập dataset nếu không có
    from datasets import Dataset
    sample_data = {
        "context": [
            "Công ty Alpha ra mắt sản phẩm X, doanh số tăng 30%.",
            "Công ty Beta bị kiện, cổ phiếu giảm 10%."
        ],
        "label": ["positive", "negative"]
    }
    dataset = Dataset.from_dict(sample_data)
    dataset.save_to_disk(args.datasets_dir)

    tuning_lm_with_rl(args)

# explain mix PPO

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from torchdiffeq import odeint # Cần cài đặt: pip install torchdiffeq
import numpy as np
import random
from trl import PPOConfig, PPOTrainer, AutoModelForCausalLMWithValueHead
from datasets import Dataset # Để tạo dataset cho PPO
import time # Để theo dõi thời gian

# Thiết lập seed
fix_seed = 100
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(fix_seed)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LLM_EXPLANATION_MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B" # Hoặc mô hình bạn muốn tinh chỉnh
LLM_GUIDANCE_MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

MAX_SEQ_LENGTH_GUIDANCE = 512 # Giảm để tiết kiệm bộ nhớ, điều chỉnh nếu cần
MAX_SEQ_LENGTH_EXPLANATION_QUERY = 256
MAX_NEW_TOKENS_EXPLANATION = 100 # Số token tối đa cho giải thích được tạo
LEARNING_RATE_FLOW_MODEL = 2e-4
LEARNING_RATE_EXPLANATION_LLM = 1.41e-5 # Thường thấp hơn cho PPO

# --- Quantization Config (Nếu cần để tiết kiệm bộ nhớ) ---
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
#     bnb_4bit_use_double_quant=True,
# )

# --- Lớp Guidance LLM (Giữ nguyên từ lần trước, có thể thêm try-except cho load model) ---
class GuidanceLLM:
    def __init__(self, model_name=LLM_GUIDANCE_MODEL_NAME):
        self.model_name = model_name
        self.tokenizer = None
        self.model = None
        self.hidden_dim = None
        self._load_model()

    def _load_model(self):
        print(f"Loading Guidance LLM: {self.model_name}...")
        try:
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name, trust_remote_code=True)
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                torch_dtype=torch.bfloat16 if DEVICE.type == "cuda" else torch.float32,
                trust_remote_code=True,
                # quantization_config=bnb_config # Bỏ comment nếu dùng quantization
            ).to(DEVICE)

            for param in self.model.parameters():
                param.requires_grad = False
            self.model.eval()

            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
                self.model.config.pad_token_id = self.model.config.eos_token_id
            # Đảm bảo padding_side cho tokenizer nếu nó sẽ được dùng để tạo input cho model này
            # self.tokenizer.padding_side = "left" # Thường cho mô hình tự hồi quy

            self.hidden_dim = self.model.config.hidden_size
            print(f"Guidance LLM loaded on {DEVICE}. Hidden dim: {self.hidden_dim}")
        except Exception as e:
            print(f"Error loading Guidance LLM: {e}")
            raise

    def get_decision_distribution(self, user_input_text, explanation_sentences_so_far, all_possible_actions):
        current_explanation = " ".join(explanation_sentences_so_far)
        action_log_probs_list = []

        with torch.no_grad():
            for action_text in all_possible_actions:
                prompt_base = (
                    f"Given the following information about a company: '{user_input_text}'.\n"
                    f"Based on the reasoning: '{current_explanation}'.\n"
                    f"Predict the price impact:"
                ) # Không bao gồm action_text trong prompt_base ban đầu
                
                inputs_base = self.tokenizer(prompt_base, return_tensors="pt", add_special_tokens=True).to(DEVICE)
                action_ids_only = self.tokenizer(action_text, add_special_tokens=False, return_tensors="pt").input_ids.to(DEVICE)

                if action_ids_only.shape[1] == 0: # Nếu action_text rỗng hoặc chỉ có special tokens
                    action_log_probs_list.append(torch.tensor(-float('inf')).to(DEVICE))
                    continue

                # Nối action_ids vào input_ids của prompt base để tạo full_input_ids
                full_input_ids = torch.cat([inputs_base.input_ids, action_ids_only], dim=1)
                
                # Giới hạn độ dài để tránh lỗi OOM
                if full_input_ids.shape[1] > MAX_SEQ_LENGTH_GUIDANCE:
                    # print(f"Warning: Prompt for action '{action_text}' too long, truncating.")
                    full_input_ids = full_input_ids[:, :MAX_SEQ_LENGTH_GUIDANCE]

                # Attention mask cần được tạo lại cho full_input_ids
                attention_mask = torch.ones_like(full_input_ids)
                
                try:
                    outputs = self.model(input_ids=full_input_ids, attention_mask=attention_mask)
                    logits = outputs.logits # (batch_size, seq_len, vocab_size)

                    # Tính log-probability của chuỗi action_ids
                    # Chúng ta cần logit của token_i tại vị trí (i-1) của action_ids
                    # bắt đầu từ vị trí của token đầu tiên của action_ids trong full_input_ids
                    action_start_index_in_logits = inputs_base.input_ids.shape[1] -1 # vị trí logit cho token đầu tiên của action
                    
                    total_log_prob = torch.tensor(0.0, device=DEVICE)
                    for i in range(action_ids_only.shape[1]):
                        logit_idx = action_start_index_in_logits + i
                        if logit_idx < 0 or logit_idx >= logits.shape[1]: # Kiểm tra biên
                            # print(f"Warning: Logit index {logit_idx} out of bounds for action '{action_text}'.")
                            total_log_prob = torch.tensor(-float('inf')).to(DEVICE)
                            break
                        
                        token_logits = logits[:, logit_idx, :]
                        log_probs_vocab = F.log_softmax(token_logits, dim=-1)
                        token_to_select = action_ids_only[:, i]
                        token_log_prob = log_probs_vocab.gather(1, token_to_select.unsqueeze(-1)).squeeze(-1)
                        total_log_prob += token_log_prob
                    
                    action_log_probs_list.append(total_log_prob.squeeze()) # Squeeze nếu batch_size=1

                except Exception as e:
                    print(f"Error processing action '{action_text}' in get_decision_distribution: {e}")
                    action_log_probs_list.append(torch.tensor(-float('inf')).to(DEVICE))


        if not action_log_probs_list or all(lp.item() == -float('inf') for lp in action_log_probs_list):
            # print("Warning: All action log_probs are -inf. Returning uniform distribution.")
            return torch.ones(len(all_possible_actions), device=DEVICE) / len(all_possible_actions)

        log_probs_tensor = torch.stack(action_log_probs_list)
        # Vì đây là log_probs, chúng ta có thể dùng softmax trực tiếp
        probability_distribution = torch.softmax(log_probs_tensor, dim=0)
        return probability_distribution

    def get_last_layer_hidden_states(self, user_input_text, explanation_sentences_so_far, all_possible_actions_text_for_prompt):
        current_explanation = " ".join(explanation_sentences_so_far)
        prompt = (
            f"Given the following information about a company: '{user_input_text}'.\n"
            f"Based on the reasoning: '{current_explanation}'.\n"
            f"Consider the possible outcomes: {all_possible_actions_text_for_prompt}."
        )
        inputs = self.tokenizer(
            prompt, return_tensors="pt", padding=True, truncation=True, max_length=MAX_SEQ_LENGTH_GUIDANCE
        ).to(DEVICE)
        with torch.no_grad():
            outputs = self.model(**inputs, output_hidden_states=True)
        return outputs.hidden_states[-1]

# --- Lớp Rectified Flow Model (Giữ nguyên từ lần trước, đã được điều chỉnh cho Qwen) ---
class RectifiedFlowModel(nn.Module):
    def __init__(
        self,
        guidance_llm_instance,
        num_actions,
        flow_embed_dim=256,
        projector_hidden_dim=256,
        projector_layers=4,
        num_attention_heads=4,
        dropout_rate=0.1
    ):
        super().__init__()
        self.guidance_llm_instance = guidance_llm_instance
        self.guidance_llm_internal_model = guidance_llm_instance.model
        self.num_actions = num_actions
        self.flow_embed_dim = flow_embed_dim
        self.projector_hidden_dim = projector_hidden_dim
        self.projector_layers = projector_layers
        self.num_attention_heads = num_attention_heads
        self.hidden_dim_guidance_llm = self.guidance_llm_instance.hidden_dim

        self.zt_embed_mlp = nn.Sequential(
            nn.Linear(num_actions, flow_embed_dim), nn.ReLU(), nn.LayerNorm(flow_embed_dim)
        )
        self.time_embed_mlp = nn.Sequential(
            nn.Linear(1, flow_embed_dim), nn.ReLU(), nn.LayerNorm(flow_embed_dim)
        )

        try:
            last_guidance_layer_attn = self.guidance_llm_internal_model.transformer.h[-1].attn
            self.guidance_wq = last_guidance_layer_attn.q_proj
            self.guidance_wk = last_guidance_layer_attn.k_proj
            self.guidance_wv = last_guidance_layer_attn.v_proj
            for param_group in [self.guidance_wq.parameters(), self.guidance_wk.parameters(), self.guidance_wv.parameters()]:
                for p in param_group: p.requires_grad = False
            self.using_borrowed_weights = True
            print("FlowModel: Successfully borrowed Q, K, V projection weights.")
        except AttributeError as e:
            print(f"FlowModel Warning: Could not borrow Q,K,V weights ({e}). Using new Linear layers.")
            self.flow_q_proj_new = nn.Linear(flow_embed_dim, self.hidden_dim_guidance_llm)
            self.flow_k_proj_new = nn.Linear(flow_embed_dim, self.hidden_dim_guidance_llm)
            self.flow_v_proj_new = nn.Linear(flow_embed_dim, self.hidden_dim_guidance_llm)
            self.guidance_k_proj_new = nn.Linear(self.hidden_dim_guidance_llm, self.hidden_dim_guidance_llm)
            self.guidance_v_proj_new = nn.Linear(self.hidden_dim_guidance_llm, self.hidden_dim_guidance_llm)
            self.using_borrowed_weights = False

        self.attention_layer = nn.MultiheadAttention(
            embed_dim=self.hidden_dim_guidance_llm, num_heads=num_attention_heads, dropout=dropout_rate, batch_first=True
        )
        self.attn_output_norm = nn.LayerNorm(self.hidden_dim_guidance_llm)
        self.attn_output_projection = nn.Linear(self.hidden_dim_guidance_llm, flow_embed_dim)

        projector_modules = []
        current_dim_proj = flow_embed_dim + num_actions + 1 # h_attn_zt + z_t + time_t
        for i in range(projector_layers):
            output_dim_proj = projector_hidden_dim if i < projector_layers - 1 else num_actions
            projector_modules.append(nn.Linear(current_dim_proj, output_dim_proj))
            if i < projector_layers - 1:
                projector_modules.extend([
                    nn.ReLU(), nn.LayerNorm(output_dim_proj), nn.Dropout(dropout_rate)
                ])
                current_dim_proj = output_dim_proj + num_actions + 1 # Cho skip connection ở lớp tiếp theo
        self.projector_mlp = nn.Sequential(*projector_modules)
        self.optimizer = optim.AdamW(self.parameters(), lr=LEARNING_RATE_FLOW_MODEL)

    def _project_for_attention(self, tensor, projection_type):
        if self.using_borrowed_weights:
            if projection_type == 'q_flow': return self.guidance_wq(tensor)
            elif projection_type == 'k_flow': return self.guidance_wk(tensor)
            elif projection_type == 'v_flow': return self.guidance_wv(tensor)
            elif projection_type == 'k_guidance': return self.guidance_wk(tensor)
            elif projection_type == 'v_guidance': return self.guidance_wv(tensor)
        else:
            if projection_type == 'q_flow': return self.flow_q_proj_new(tensor)
            elif projection_type == 'k_flow': return self.flow_k_proj_new(tensor)
            elif projection_type == 'v_flow': return self.flow_v_proj_new(tensor)
            elif projection_type == 'k_guidance': return self.guidance_k_proj_new(tensor)
            elif projection_type == 'v_guidance': return self.guidance_v_proj_new(tensor)
        raise ValueError(f"Unknown projection_type: {projection_type}")

    def forward(self, z_t, time_t, guidance_llm_last_hidden_states):
        h_emb_zt = self.zt_embed_mlp(z_t)
        h_emb_t = self.time_embed_mlp(time_t)
        flow_tokens_emb_stacked = torch.stack([h_emb_zt, h_emb_t], dim=1)

        query_input_tensor = flow_tokens_emb_stacked[:, 0:1, :]
        q_projected = self._project_for_attention(query_input_tensor, 'q_flow')

        k_flow_projected = self._project_for_attention(flow_tokens_emb_stacked, 'k_flow')
        k_guidance_projected = self._project_for_attention(guidance_llm_last_hidden_states, 'k_guidance')
        combined_keys = torch.cat([k_flow_projected, k_guidance_projected], dim=1)

        v_flow_projected = self._project_for_attention(flow_tokens_emb_stacked, 'v_flow')
        v_guidance_projected = self._project_for_attention(guidance_llm_last_hidden_states, 'v_guidance')
        combined_values = torch.cat([v_flow_projected, v_guidance_projected], dim=1)

        attn_output, _ = self.attention_layer(query=q_projected, key=combined_keys, value=combined_values)
        attn_output_normalized = self.attn_output_norm(attn_output)
        h_attn_zt = self.attn_output_projection(attn_output_normalized.squeeze(1))

        projector_input_full = torch.cat([h_attn_zt, z_t, time_t], dim=1)
        vector_field = self.projector_mlp(projector_input_full)
        return vector_field

    def train_step(self, z0_batch, z1_batch, time_batch, guidance_llm_hidden_states_batch):
        self.optimizer.zero_grad()
        z_t_batch = time_batch * z1_batch + (1.0 - time_batch) * z0_batch
        target_vector_field = z1_batch - z0_batch
        predicted_vector_field = self.forward(z_t_batch, time_batch, guidance_llm_hidden_states_batch)
        loss = nn.MSELoss()(predicted_vector_field, target_vector_field)
        loss.backward()
        self.optimizer.step()
        return loss.item()

    @torch.no_grad()
    def solve_ode_generate_distribution(self, z0, guidance_llm_last_hidden_states, num_steps=10, return_logits=False):
        self.eval()
        if z0.ndim == 1: z0 = z0.unsqueeze(0)
        if guidance_llm_last_hidden_states.ndim == 2: guidance_llm_last_hidden_states = guidance_llm_last_hidden_states.unsqueeze(0)
        batch_size = z0.shape[0]
        def ode_func(t_scalar, z_flat): # t là scalar, z_flat là [B*N]
            z = z_flat.view(batch_size, -1) # Reshape về [B, N]
            # Tạo time_t_tensor với batch_size
            time_t_tensor = torch.full((batch_size, 1), t_scalar.item(), device=z.device, dtype=z.dtype)
            vf = self.forward(z, time_t_tensor, guidance_llm_last_hidden_states)
            return vf.flatten() # Flatten lại cho ODE solver

        t_span = torch.linspace(0, 1, num_steps, device=z0.device)
        z_t_solutions = odeint(ode_func, z0.flatten(), t_span, method='rk4') # rk4, dopri5
        z1_hat_flat = z_t_solutions[-1]
        z1_hat = z1_hat_flat.view(batch_size, -1) # Reshape lại

        self.train()
        output = z1_hat.squeeze(0) if batch_size == 1 else z1_hat
        return output if return_logits else torch.softmax(output, dim=-1)

# --- Lớp Explanation LLM với PPO ---
class ExplanationLLM_PPO:
    def __init__(self, model_name, ppo_config):
        self.model_name = model_name
        self.ppo_config = ppo_config

        # Load model với ValueHead cho PPO
        self.model = AutoModelForCausalLMWithValueHead.from_pretrained(
            model_name,
            trust_remote_code=True,
            # quantization_config=bnb_config, # Nếu dùng quantization
            torch_dtype=torch.bfloat16 if DEVICE.type == "cuda" else torch.float32,
            # peft_config=lora_config, # Nếu dùng LoRA
        ).to(DEVICE) # Cần .to(DEVICE) ở đây

        self.tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        self.tokenizer.padding_side = "left" # Quan trọng cho PPO với mô hình tự hồi quy

        self.ppo_trainer = PPOTrainer(
            config=ppo_config,
            model=self.model,
            tokenizer=self.tokenizer
            # optimizer, lr_scheduler có thể được truyền vào nếu bạn muốn tùy chỉnh
        )
        print(f"Explanation LLM (for PPO) loaded: {model_name}")


    def generate_explanation_batch(self, query_tensors_list, generation_kwargs):
        """ Tạo giải thích cho một batch các query tensors """
        # query_tensors_list: list of torch.Tensor (mỗi tensor là tokenized query)
        # response_tensors_list = self.ppo_trainer.generate(
        #     query_tensors_list,
        #     return_prompt=False, # Chỉ trả về phần được sinh ra
        #     length_sampler=None, # Hoặc một length sampler nếu muốn độ dài thay đổi
        #     **generation_kwargs
        # )
        # return response_tensors_list
        # TRL PPOTrainer.generate mong muốn một list of single query_tensors (không phải batch)
        # hoặc một batch tensor duy nhất. Ở đây ta sẽ truyền từng query một.
        
        response_tensors_list = []
        for query_tensor in query_tensors_list:
            # Đảm bảo query_tensor có batch_dim = 1
            if query_tensor.ndim == 1:
                query_tensor = query_tensor.unsqueeze(0)
            response = self.ppo_trainer.generate(
                query_tensor,
                return_prompt=False,
                **generation_kwargs
            ) # response sẽ là [1, seq_len]
            response_tensors_list.append(response.squeeze(0)) # Bỏ batch_dim
        return response_tensors_list


    def decode_responses(self, response_tensors_list):
        return [self.tokenizer.decode(r, skip_special_tokens=True) for r in response_tensors_list]

    def train_step(self, query_tensors, response_tensors, reward_tensors):
        # query_tensors: list of 1D tensors
        # response_tensors: list of 1D tensors
        # reward_tensors: list of scalar tensors
        # PPOTrainer.step() sẽ xử lý việc tính toán loss và cập nhật model
        stats = self.ppo_trainer.step(query_tensors, response_tensors, reward_tensors)
        return stats

# --- Vòng lặp Huấn luyện Chính ---
def main_training_loop(
    dataset_list_of_dicts, # list of {'user_input': ..., 'prediction': ..., 'explain': ...}
    possible_actions,
    num_rounds=2,
    epochs_flow_model=20, # Ít hơn để test nhanh
    ppo_epochs_per_round=1, # Số lần lặp qua PPO data trong 1 round
    ppo_batch_size=4,
    gradient_accumulation_steps_ppo=1
    ):

    print("--- Initializing Models ---")
    guidance_llm = GuidanceLLM(model_name=LLM_GUIDANCE_MODEL_NAME)
    
    ppo_config = PPOConfig(
        model_name=LLM_EXPLANATION_MODEL_NAME,
        learning_rate=LEARNING_RATE_EXPLANATION_LLM,
        batch_size=ppo_batch_size,
        mini_batch_size=ppo_batch_size // 2 if ppo_batch_size > 1 else 1, # Phải <= batch_size
        gradient_accumulation_steps=gradient_accumulation_steps_ppo,
        ppo_epochs=4, # Số PPO epochs mỗi khi gọi step
        log_with="wandb", # hoặc None, "tensorboard"
        # Các siêu tham số PPO khác: kl_penalty, target_kl, clip_epsilon, vf_coef, etc.
        remove_unused_columns=False, # Quan trọng nếu dataset có cột thừa
    )
    explanation_llm_ppo = ExplanationLLM_PPO(LLM_EXPLANATION_MODEL_NAME, ppo_config)

    flow_model = RectifiedFlowModel(
        guidance_llm_instance=guidance_llm,
        num_actions=len(possible_actions),
        flow_embed_dim=128, projector_hidden_dim=128, num_attention_heads=2
    ).to(DEVICE)
    print("--- Models Initialized ---")

    generation_kwargs_expl = {
        "max_new_tokens": MAX_NEW_TOKENS_EXPLANATION,
        "min_length": -1, # Không có min_length
        "top_k": 0.0,
        "top_p": 1.0,
        "do_sample": True,
        "pad_token_id": explanation_llm_ppo.tokenizer.pad_token_id,
        "eos_token_id": explanation_llm_ppo.tokenizer.eos_token_id,
    }

    for round_num in range(num_rounds):
        print(f"\n=============== ROUND {round_num + 1}/{num_rounds} ===============")
        start_time_round = time.time()

        # --- Giai đoạn 1: Tạo Mẫu Dương cho Flow Model ---
        print(f"\n--- Round {round_num+1}: Generating Positive Samples for Flow Model ---")
        positive_samples_for_flow = [] # list of (z0, z1, guidance_hidden_states)
        num_positive_found = 0
        for i, data_sample in enumerate(dataset_list_of_dicts):
            if i % 50 == 0: print(f"  Processing data sample {i}/{len(dataset_list_of_dicts)} for positive samples...")
            user_input = data_sample['user_input']
            actual_expert_pred = data_sample['prediction']
            # Sử dụng giải thích "vàng" từ dataset để tạo mẫu dương chất lượng cao ban đầu
            expert_explanation_sents = [s.strip() for s in data_sample['explain'].split('.') if s.strip() and len(s.strip()) > 3]
            if not expert_explanation_sents: continue

            prob_dist_guidance = guidance_llm.get_decision_distribution(
                user_input, expert_explanation_sents, possible_actions
            )
            pred_idx_guidance = torch.argmax(prob_dist_guidance)

            if possible_actions[pred_idx_guidance] == actual_expert_pred:
                num_positive_found += 1
                z1 = prob_dist_guidance.detach().cpu()
                z0 = torch.randn_like(z1)
                actions_prompt_text = f"[{', '.join(possible_actions)}]"
                guidance_hs = guidance_llm.get_last_layer_hidden_states(
                    user_input, expert_explanation_sents, actions_prompt_text
                ).detach().cpu().squeeze(0) # Bỏ batch dim
                positive_samples_for_flow.append((z0, z1, guidance_hs))
        
        if not positive_samples_for_flow:
            print("No positive samples found for Flow Model in this round. Skipping Flow Model training.")
        else:
            print(f"Found {num_positive_found} positive samples for Flow Model.")
            # --- Giai đoạn 2: Huấn luyện Rectified Flow Model (φ) ---
            print(f"\n--- Round {round_num+1}: Training Rectified Flow Model (φ) ---")
            flow_model.train()
            for epoch in range(epochs_flow_model):
                epoch_start_time = time.time()
                total_flow_loss = 0
                random.shuffle(positive_samples_for_flow) # Xáo trộn mỗi epoch
                # Implement batching for flow model training
                flow_batch_size = 32
                for batch_idx in range(0, len(positive_samples_for_flow), flow_batch_size):
                    batch_samples = positive_samples_for_flow[batch_idx : batch_idx + flow_batch_size]
                    if not batch_samples: continue

                    z0_b = torch.stack([s[0] for s in batch_samples]).to(DEVICE)
                    z1_b = torch.stack([s[1] for s in batch_samples]).to(DEVICE)
                    guidance_hs_b = torch.stack([s[2] for s in batch_samples]).to(DEVICE)
                    time_b = torch.rand(z0_b.shape[0], 1, device=DEVICE) # (batch_size, 1)

                    loss = flow_model.train_step(z0_b, z1_b, time_b, guidance_hs_b)
                    total_flow_loss += loss * z0_b.shape[0] # Tính loss có trọng số batch size
                avg_flow_loss = total_flow_loss / len(positive_samples_for_flow)
                epoch_time = time.time() - epoch_start_time
                print(f"  Flow Model Epoch {epoch+1}/{epochs_flow_model}, Avg Loss: {avg_flow_loss:.6f}, Time: {epoch_time:.2f}s")


        # --- Giai đoạn 3: Huấn luyện EXPLANATION LLM (πε) bằng PPO ---
        print(f"\n--- Round {round_num+1}: Training Explanation LLM (πε) with PPO ---")
        explanation_llm_ppo.model.train() # Đảm bảo model ở chế độ train cho PPO
        
        # Chuẩn bị dữ liệu cho PPO (lặp qua dataset để tạo queries)
        ppo_data_list = [] # List of dicts: {'query': str, 'input_ids': tensor, 'actual_pred': str, 'user_input':str}
        for data_sample in dataset_list_of_dicts:
            # Query cho Explanation LLM là user_input
            query_prompt = f"Given the context: '{data_sample['user_input']}'. Please analyze the reasoning for the predicted outcome."
            # Tokenize query
            tokenized_query = explanation_llm_ppo.tokenizer(
                query_prompt,
                truncation=True,
                max_length=MAX_SEQ_LENGTH_EXPLANATION_QUERY, # Giới hạn độ dài query
                return_tensors="pt"
            ).input_ids.squeeze(0).to(DEVICE) # Bỏ batch dim, lên device

            ppo_data_list.append({
                "query_text": query_prompt, # Để debug
                "query_tensor": tokenized_query,
                "actual_pred_text": data_sample['prediction'],
                "user_input_text_for_guidance": data_sample['user_input'] # Để truyền cho guidance/flow
            })
        
        # PPO training loop
        for ppo_epoch_num in range(ppo_epochs_per_round): # Có thể lặp PPO nhiều lần trên cùng 1 batch dữ liệu được tạo ra
            print(f"  PPO Epoch {ppo_epoch_num + 1}/{ppo_epochs_per_round}")
            ppo_epoch_start_time = time.time()
            # Xáo trộn dữ liệu PPO
            random.shuffle(ppo_data_list)

            # Tạo batch cho PPO
            for batch_start_idx in range(0, len(ppo_data_list), ppo_config.batch_size):
                batch_end_idx = batch_start_idx + ppo_config.batch_size
                ppo_batch = ppo_data_list[batch_start_idx:batch_end_idx]
                if not ppo_batch: continue

                batch_query_tensors = [item['query_tensor'] for item in ppo_batch]
                
                # 1. Explanation LLM tạo giải thích (response)
                response_tensors = explanation_llm_ppo.generate_explanation_batch(
                    batch_query_tensors, generation_kwargs_expl
                ) # List of 1D tensors

                batch_rewards = []
                for i in range(len(ppo_batch)):
                    query_item = ppo_batch[i]
                    generated_response_tensor = response_tensors[i]
                    generated_explanation_text = explanation_llm_ppo.tokenizer.decode(generated_response_tensor, skip_special_tokens=True)
                    generated_sents = [s.strip() for s in generated_explanation_text.split('.') if s.strip() and len(s.strip()) > 3]
                    
                    if not generated_sents: # Nếu không tạo được câu nào có nghĩa
                        batch_rewards.append(torch.tensor(-1.0, device=DEVICE)) # Phần thưởng phạt nặng
                        continue

                    # 2. Tính phần thưởng cho giải thích này bằng Flow Model
                    # Phần thưởng theo từng câu, sau đó lấy phần thưởng cuối cùng hoặc tổng hợp
                    # p_hat_prev_sentence = # p_hat khi chỉ có context (k=0)
                    # Để tính p_hat(k=0), ta cần hidden_states khi explanation rỗng
                    hs_k0 = guidance_llm.get_last_layer_hidden_states(
                        query_item['user_input_text_for_guidance'], [], f"[{', '.join(possible_actions)}]"
                    ).to(DEVICE)
                    z0_phi = torch.randn(len(possible_actions), device=DEVICE)
                    p_hat_k_minus_1_dist = flow_model.solve_ode_generate_distribution(z0_phi, hs_k0)
                    
                    actual_pred_idx = possible_actions.index(query_item['actual_pred_text'])
                    p_hat_val_k_minus_1 = p_hat_k_minus_1_dist[actual_pred_idx].item()
                    
                    final_reward_for_explanation = 0.0
                    current_cumulative_sents = []
                    for sent_idx, sent_text in enumerate(generated_sents):
                        current_cumulative_sents.append(sent_text)
                        hs_current_sent = guidance_llm.get_last_layer_hidden_states(
                            query_item['user_input_text_for_guidance'], current_cumulative_sents, f"[{', '.join(possible_actions)}]"
                        ).to(DEVICE)
                        
                        p_hat_current_sent_dist = flow_model.solve_ode_generate_distribution(z0_phi, hs_current_sent)
                        p_hat_val_current_sent = p_hat_current_sent_dist[actual_pred_idx].item()
                        
                        # Reward cho câu này
                        # reward_this_sentence = p_hat_val_current_sent - p_hat_val_k_minus_1
                        # p_hat_val_k_minus_1 = p_hat_val_current_sent
                        
                        # Theo bài báo, reward cho cả giải thích là thay đổi p_hat sau câu cuối cùng
                        if sent_idx == len(generated_sents) - 1:
                             final_reward_for_explanation = p_hat_val_current_sent - p_hat_val_k_minus_1 # Hoặc p_hat_val_k0
                                                                                                       # nếu reward là tổng gain
                    batch_rewards.append(torch.tensor(final_reward_for_explanation, device=DEVICE))

                # 3. Huấn luyện Explanation LLM bằng PPO step
                # PPOTrainer.step nhận list of tensors cho query, response, reward
                stats = explanation_llm_ppo.train_step(batch_query_tensors, response_tensors, batch_rewards)
                
                if stats and 'ppo/loss/total' in stats: # Kiểm tra stats không rỗng
                    print(f"    PPO Batch {batch_start_idx // ppo_config.batch_size +1}, Loss: {stats['ppo/loss/total']:.4f}, Mean Reward: {stats['ppo/mean_scores']:.4f}")
            
            ppo_epoch_time = time.time() - ppo_epoch_start_time
            print(f"  PPO Epoch {ppo_epoch_num + 1} finished in {ppo_epoch_time:.2f}s.")

        round_time = time.time() - start_time_round
        print(f"=============== ROUND {round_num + 1} completed in {round_time:.2f}s ===============")

    print("--- Training Finished ---")


# --- Dữ liệu ví dụ (cần nhiều hơn cho huấn luyện thực tế) ---
sample_dataset = [
    {'user_input': "Công ty A: Báo cáo lợi nhuận quý 3 vượt kỳ vọng 15%. CEO công bố kế hoạch mở rộng sang thị trường Châu Á. Chỉ số P/E hiện tại là 12, thấp hơn trung bình ngành (15).",
     'prediction': "positive",
     'explain': "Lợi nhuận vượt trội là tín hiệu mạnh. Kế hoạch mở rộng cho thấy tiềm năng tăng trưởng. P/E hấp dẫn so với ngành củng cố quan điểm tích cực."},
    {'user_input': "Công ty B: Đối mặt với vụ kiện tụng lớn có thể ảnh hưởng đến 20% doanh thu. Giá cổ phiếu đã giảm 10% trong tuần qua. Ban lãnh đạo chưa có thông báo chính thức.",
     'prediction': "negative",
     'explain': "Vụ kiện tụng là rủi ro nghiêm trọng. Sự im lặng của ban lãnh đạo gây thêm bất ổn. Khả năng sụt giảm doanh thu là đáng kể."},
    {'user_input': "Công ty C: Ra mắt sản phẩm Z được đánh giá cao nhưng chi phí marketing tăng vọt. Tỷ lệ nợ trên vốn chủ sở hữu ở mức cao (1.5). Thị trường cạnh tranh gay gắt.",
     'prediction': "negative", # Hoặc có thể là 'neutral' nếu có
     'explain': "Chi phí marketing lớn và nợ cao tạo áp lực tài chính. Mặc dù sản phẩm mới tốt, rủi ro tổng thể vẫn nghiêng về tiêu cực trong môi trường cạnh tranh này."},
    # Thêm nhiều mẫu hơn...
    {'user_input': "Công ty D: Vừa nhận được khoản đầu tư lớn từ quỹ uy tín. Công nghệ lõi được cấp bằng sáng chế độc quyền. Đang tuyển dụng nhiều vị trí kỹ sư cao cấp.",
     'prediction': "positive",
     'explain': "Khoản đầu tư lớn xác nhận tiềm năng. Bằng sáng chế tạo lợi thế cạnh tranh. Việc tuyển dụng nhân sự giỏi cho thấy sự chuẩn bị cho tăng trưởng."},
    {'user_input': "Công ty E: Doanh thu đi ngang trong 3 quý liên tiếp. Thị phần đang bị các đối thủ mới nổi gặm nhấm. Cắt giảm chi phí nhân sự được công bố.",
     'prediction': "negative",
     'explain': "Doanh thu trì trệ và mất thị phần là dấu hiệu xấu. Cắt giảm nhân sự cho thấy khó khăn nội tại. Triển vọng không mấy khả quan."},
]
POSSIBLE_ACTIONS = ["negative", "positive"]


if __name__ == '__main__':
    # Chạy vòng lặp huấn luyện chính
    # Cần nhiều dữ liệu hơn và thời gian huấn luyện dài hơn cho kết quả tốt
    main_training_loop(
        dataset_list_of_dicts=sample_dataset * 20, # Lặp lại dataset để có nhiều mẫu hơn cho demo
        possible_actions=POSSIBLE_ACTIONS,
        num_rounds=2,
        epochs_flow_model=5, # Giảm để chạy nhanh hơn trong demo
        ppo_epochs_per_round=2,
        ppo_batch_size=2, # Giảm batch size cho demo
        gradient_accumulation_steps_ppo=1
    )

# tuning PPO OLD

In [ ]:
def tuning_lm_with_rl(args):
    # Khởi tạo script_args từ args
    script_args = args
    reward_model_name = script_args.reward_model_name
    print("reward_model_name:", reward_model_name)

    # Đường dẫn dataset
    dataset_name = script_args.datasets_dir
    print("dataset_name:", dataset_name)

    # Cấu hình PPO
    config = PPOConfig(
        learning_rate=script_args.rl_learning_rate,
        # batch_size=script_args.batch_size,
        batch_size=4,
        # mini_batch_size=script_args.mini_batch_size,
        mini_batch_size=2,
        # gradient_accumulation_steps=script_args.rl_gradient_accumulation_steps,
        gradient_accumulation_steps=1,
        ppo_epochs=script_args.ppo_epochs,  
        seed=script_args.seed,
    )

    # Tên mô hình gốc
    model_name = script_args.rl_base_model
    print("model_name:", model_name)

    # Tải dataset
    train_dataset = load_dataset(dataset_name, split="train")
    print("train_dataset size:", len(train_dataset))

    # Cấu hình tham số cho sentiment pipeline
    sent_kwargs = {
        "return_all_scores": True,
        "function_to_apply": "none",
        "batch_size": 1,
        "truncation": True
    }

    # Tải tokenizer
    tokenizer = AutoTokenizer.from_pretrained(script_args.tokenizer_name, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    print("Tokenizer loaded:", tokenizer.__class__.__name__)

    def build_dataset(tokenizer, dataset_name, input_min_text_length=2, input_max_text_length=8):
        """
        Tạo dataset cho huấn luyện PPO.
        
        Args:
            tokenizer: Tokenizer để mã hóa văn bản.
            dataset_name: Tên hoặc đường dẫn dataset.
            input_min_text_length: Độ dài tối thiểu của câu hỏi.
            input_max_text_length: Độ dài tối đa của câu hỏi.
        
        Returns:
            Dataset đã được xử lý với các cột query và input_ids.
        """
        ds = load_dataset(dataset_name, split="train")
        # Giả sử dataset của bạn tên là `ds`, hiện có 6 dòng
        repeat_factor = 64 // len(ds) + 1  # Lặp đủ số lần để vượt 64

        # Lặp lại nhiều lần rồi cắt còn đúng 64
        ds = concatenate_datasets([ds] * repeat_factor)
        ds = ds.select(range(64))  # Lấy đúng 64 dòng
        original_columns = ds.column_names

        def preprocess_function(examples):
            new_examples = {
                "query": [],
                "input_ids": [],
            }
            # Giả định dataset có cột 'user_input' hoặc 'question'
            input_key = "user_input" if "user_input" in examples else "question"
            for question in examples[input_key]:
                query = "Question: " + question + "\n\nAnswer: "
                tokenized_question = tokenizer(
                    query,
                    truncation=True,
                    max_length=512,
                    return_tensors="pt"
                )
                new_examples["query"].append(query)
                new_examples["input_ids"].append(tokenized_question["input_ids"].squeeze(0))
            return new_examples

        ds = ds.map(
            preprocess_function,
            batched=True,
            num_proc=1,
            remove_columns=original_columns,
        )
        ds.set_format(type="torch")
        return ds

    # Tạo dataset
    dataset = build_dataset(tokenizer, dataset_name=dataset_name)
    print("Dataset created with", len(dataset), "samples")

    def collator(data):
        """
        Collator để xử lý batch dữ liệu.
        """
        return dict((key, [d[key] for d in data]) for key in data[0])
    

    # Đặt seed để đảm bảo tính tái lập
    torch.manual_seed(config.seed)

    # Cấu hình LoRA
    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
    )

    # Tải mô hình chính với value head
    model = AutoModelForCausalLMWithValueHead.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,  # Đồng bộ với reward model
        peft_config=lora_config,
    )

    # Gán thủ công base_model_prefix để tránh lỗi
    model.base_model_prefix = "model"  # hoặc "transformer", tùy thuộc vào tên trong mô hình gốc
    # Gán generation_config để tránh lỗi AttributeError
    model.generation_config = GenerationConfig.from_pretrained(
        "./saved_models/lora-DeepSeek-R1-Distill-Qwen-adapter-merged"
    )
    # model.config.return_dict = True

    # Tạo generation_config nếu không có
    if not hasattr(model, "generation_config"):
        model.generation_config = GenerationConfig.from_pretrained(model_name, trust_remote_code=True)

    print("Finetune model:", model_name, type(model))

    
    # Tạo optimizer (nếu dùng Adafactor)
    optimizer = None
    if script_args.adafactor:
        optimizer = Adafactor(
            filter(lambda p: p.requires_grad, model.parameters()),
            scale_parameter=False,
            relative_step=False,
            warmup_init=False,
            lr=config.learning_rate,
        )

    # Khởi tạo PPOTrainer
    print(dataset)
    ppo_trainer = PPOTrainer(
        config=config,  # Sử dụng args thay vì config
        model=model,
        tokenizer=tokenizer,  
        dataset=dataset,
        data_collator=collator,
        optimizer=optimizer,
    )
    
    # ppo_trainer.train()

    # Xác định thiết bị
    device = ppo_trainer.accelerator.device
    if ppo_trainer.accelerator.num_processes == 1:
        device = 0 if torch.cuda.is_available() else "cpu"
    print("Device:", device)

    # Tạo sentiment pipeline
    sentiment_pipe = pipeline(
        "sentiment-analysis",
        model=reward_model_name,
        device_map="auto",
        # config=reward_model_config,
        tokenizer=tokenizer,
        # device=device,
    )

    # Cấu hình tham số sinh văn bản
    generation_kwargs = {
        "top_k": 0.0,
        "top_p": 1.0,
        "do_sample": True,
        "pad_token_id": tokenizer.pad_token_id,
        "eos_token_id": tokenizer.eos_token_id,
    }
    output_min_length = 32
    output_max_length = script_args.output_max_length
    output_length_sampler = LengthSampler(output_min_length, output_max_length)

    # Vòng lặp huấn luyện PPO
    for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader)):
        question_tensors = batch["input_ids"]

        # Sinh phản hồi
        response_tensors = ppo_trainer.generate(
            question_tensors,
            return_prompt=False,
            length_sampler=output_length_sampler,
            **generation_kwargs,
        )
        batch["response"] = tokenizer.batch_decode(response_tensors, skip_special_tokens=True)

        # Tính điểm thưởng từ reward model
        texts = [q + r for q, r in zip(batch["query"], batch["response"])]
        pipe_outputs = sentiment_pipe(texts, **sent_kwargs)
        rewards = [torch.tensor(output[0]["score"] - script_args.reward_baseline) for output in pipe_outputs]

        # Thực hiện bước PPO
        stats = ppo_trainer.step(question_tensors, response_tensors, rewards)
        ppo_trainer.log_stats(stats, batch, rewards)

        # Lưu checkpoint định kỳ
        if script_args.save_freq and epoch and epoch % script_args.save_freq == 0:
            save_dir = os.path.join(script_args.output_dir, f"step_{epoch}")
            ppo_trainer.save_pretrained(save_dir)
            print(f"Saved checkpoint at: {save_dir}")

    # Lưu checkpoint cuối cùng
    final_save_dir = os.path.join(script_args.output_dir, "step_saved")
    ppo_trainer.save_pretrained(final_save_dir)
    print(f"Final checkpoint saved at: {final_save_dir}")